# Loop 5 Strategic Analysis

## Current Status
- Best CV: 70.616068 (hybrid ensemble)
- Best LB: 70.622435 (sa_fast_v2)
- Target: 68.891380
- Gap: 1.725 points (2.50%)
- Submissions: 4/100 used, 96 remaining

## Key Insights from Research
1. SA is the dominant approach
2. N < 58: SA for unstructured packings
3. N >= 58: Tessellation/lattice is mathematically superior
4. Top teams use C++ optimizers
5. The gap of 1.725 points is HUGE - need fundamentally different approach

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
import os

os.chdir('/home/code')

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def get_scores_by_n(df):
    scores = {}
    for n in range(1, 201):
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        if len(group) != n:
            continue
        xs = group['x'].str[1:].astype(float).values
        ys = group['y'].str[1:].astype(float).values
        degs = group['deg'].str[1:].astype(float).values
        scores[n] = score_group(xs, ys, degs, TX, TY)
    return scores

print('Functions defined')

Functions defined


In [2]:
# Load our best solution and analyze per-N scores
df = pd.read_csv('/home/submission/submission.csv')
scores = get_scores_by_n(df)
total = sum(scores.values())
print(f'Total score: {total:.6f}')
print(f'Target: 68.891380')
print(f'Gap: {total - 68.891380:.6f}')

Total score: 70.616068
Target: 68.891380
Gap: 1.724688


In [3]:
# Analyze score distribution by N ranges
ranges = [
    (1, 10, 'Small (1-10)'),
    (11, 30, 'Medium-Small (11-30)'),
    (31, 57, 'Medium (31-57)'),
    (58, 100, 'Large (58-100)'),
    (101, 200, 'Very Large (101-200)')
]

print('Score contribution by N range:')
print('=' * 60)
for start, end, name in ranges:
    range_score = sum(scores[n] for n in range(start, end+1) if n in scores)
    pct = range_score / total * 100
    print(f'{name:25s}: {range_score:8.4f} ({pct:5.1f}%)')
print('=' * 60)
print(f'Total: {total:.6f}')

Score contribution by N range:
Small (1-10)             :   4.3291 (  6.1%)
Medium-Small (11-30)     :   7.4129 ( 10.5%)
Medium (31-57)           :   9.7959 ( 13.9%)
Large (58-100)           :  15.1012 ( 21.4%)
Very Large (101-200)     :  33.9769 ( 48.1%)
Total: 70.616068


In [4]:
# Find N values with highest individual scores (most room for improvement)
print('\nTop 20 N values by score contribution:')
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    pct = score / total * 100
    print(f'N={n:3d}: {score:.6f} ({pct:.2f}%)')


Top 20 N values by score contribution:
N=  1: 0.661250 (0.94%)
N=  2: 0.450779 (0.64%)
N=  3: 0.434745 (0.62%)
N=  5: 0.416850 (0.59%)
N=  4: 0.416545 (0.59%)
N=  7: 0.399897 (0.57%)
N=  6: 0.399610 (0.57%)
N=  9: 0.387415 (0.55%)
N=  8: 0.385407 (0.55%)
N= 15: 0.376949 (0.53%)
N= 10: 0.376630 (0.53%)
N= 21: 0.376451 (0.53%)
N= 20: 0.376057 (0.53%)
N= 22: 0.375258 (0.53%)
N= 11: 0.374924 (0.53%)
N= 16: 0.374128 (0.53%)
N= 26: 0.373997 (0.53%)
N= 12: 0.372724 (0.53%)
N= 13: 0.372294 (0.53%)
N= 25: 0.372144 (0.53%)


In [5]:
# Calculate theoretical minimum (perfect packing efficiency)
# Tree area = 0.2475 (calculated from polygon)
tree_area = 0.2475

print('\nTheoretical analysis:')
print('=' * 60)
print(f'Tree polygon area: {tree_area}')
print()
print('Theoretical minimum (100% packing efficiency):')
theoretical_min = 0
for n in range(1, 201):
    # Minimum square side = sqrt(n * tree_area)
    min_side = math.sqrt(n * tree_area)
    min_score = min_side ** 2 / n
    theoretical_min += min_score

print(f'Theoretical minimum score: {theoretical_min:.6f}')
print(f'Current score: {total:.6f}')
print(f'Gap to theoretical: {total - theoretical_min:.6f}')
print(f'Target: 68.891380')
print(f'Gap to target: {total - 68.891380:.6f}')


Theoretical analysis:
Tree polygon area: 0.2475

Theoretical minimum (100% packing efficiency):
Theoretical minimum score: 49.500000
Current score: 70.616068
Gap to theoretical: 21.116068
Target: 68.891380
Gap to target: 1.724688


In [6]:
# Analyze packing efficiency by N
print('\nPacking efficiency by N (sample):')
print('=' * 60)
for n in [1, 5, 10, 20, 50, 100, 150, 200]:
    if n in scores:
        actual_area = scores[n] * n  # s^2
        theoretical_area = n * tree_area
        efficiency = theoretical_area / actual_area * 100
        print(f'N={n:3d}: efficiency={efficiency:5.1f}%, score={scores[n]:.6f}')


Packing efficiency by N (sample):
N=  1: efficiency= 37.4%, score=0.661250
N=  5: efficiency= 59.4%, score=0.416850
N= 10: efficiency= 65.7%, score=0.376630
N= 20: efficiency= 65.8%, score=0.376057
N= 50: efficiency= 68.6%, score=0.360753
N=100: efficiency= 72.1%, score=0.343395
N=150: efficiency= 73.4%, score=0.337064
N=200: efficiency= 73.3%, score=0.337549


In [7]:
# Check what optimizers are available in snapshots
import subprocess

print('Available optimizers in snapshots:')
result = subprocess.run(['find', '/home/nonroot/snapshots', '-type', 'f', '-executable', '-name', '*'], 
                       capture_output=True, text=True)
for line in result.stdout.strip().split('\n')[:30]:
    if line:
        print(f'  {line}')

Available optimizers in snapshots:
  /home/nonroot/snapshots/santa-2025/21116303805/code/bbox3
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/sa_v1_parallel
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/tree_packer_v18
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/tree_packer_v21
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/002_preoptimized/bbox3
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bbox3
  /home/nonroot/snapshots/santa-2025/21328309254/code/bbox3
  /home/nonroot/snapshots/santa-2025/21328309254/code/sa_fast_v2
  /home/nonroot/snapshots/santa-2025/21328309254/code/preoptimized/bbox3
  /home/nonroot/snapshots/santa-2025/21165872902/code/bbox3
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v21
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v18
  /home/nonroot/snapshots/santa-2025/

In [8]:
# Check for sa_v1_parallel which we haven't tried yet
result = subprocess.run(['find', '/home/nonroot/snapshots', '-name', 'sa_v1_parallel*', '-o', '-name', 'sa_parallel*'], 
                       capture_output=True, text=True)
print('sa_v1_parallel locations:')
for line in result.stdout.strip().split('\n'):
    if line:
        print(f'  {line}')

sa_v1_parallel locations:
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/sa_v1_parallel
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/sa_v1_parallel.cpp
  /home/nonroot/snapshots/santa-2025/21198893057/code/experiments/013_long_sa/sa_v1_parallel_long.cpp
  /home/nonroot/snapshots/santa-2025/21198893057/code/experiments/013_long_sa/sa_v1_parallel_long
  /home/nonroot/snapshots/santa-2025/21198893057/code/exploration/datasets/sa_v1_parallel
  /home/nonroot/snapshots/santa-2025/21198893057/code/exploration/datasets/sa_v1_parallel.cpp
  /home/nonroot/snapshots/santa-2025/21190224310/code/experiments/013_long_sa/sa_v1_parallel_long.cpp
  /home/nonroot/snapshots/santa-2025/21190224310/code/experiments/013_long_sa/sa_v1_parallel_long
  /home/nonroot/snapshots/santa-2025/21190224310/code/exploration/datasets/sa_v1_parallel
  /home/nonroot/snapshots/santa-2025/21190224310/code/exploration/datasets/sa_v1_parallel.cpp
  

In [9]:
# Check for tree_packer variants
result = subprocess.run(['find', '/home/nonroot/snapshots', '-name', 'tree_packer*'], 
                       capture_output=True, text=True)
print('tree_packer locations:')
for line in result.stdout.strip().split('\n')[:20]:
    if line:
        print(f'  {line}')

tree_packer locations:
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/tree_packer_v18.cpp
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel/tree_packer_v18
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/tree_packer_v21
  /home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation/tree_packer_v21.cpp
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v18.cpp
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v21
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v18
  /home/nonroot/snapshots/santa-2025/21165872902/code/tree_packer_v21.cpp
  /home/nonroot/snapshots/santa-2025/21198893057/code/exploration/datasets/tree_packer_output.csv
  /home/nonroot/snapshots/santa-2025/21198893057/code/exploration/datasets/seowoohyeon/tree_packer
  /home/nonroot/snapshots/santa-2025/21198893057/code/exploration/da